In [435]:
%%R
install.packages('dynlm')
dyn.load('../bekk_filter/C/bekk_log_lik.so')
source('../bekk_filter/R/bekk_model.R')

In [335]:
%load_ext autoreload
%autoreload 2
import pycountry
import numpy as np
import pandas as pd
import xarray as xr
import xgboost as xgb
import seaborn as sns
import itertools as it
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.vector_ar.dynamic import DynamicPanelVAR
%run ../src/models/geo_helpers.py
%run ../src/models/section_series.py
%load_ext rpy2.ipython
%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [346]:
city_locations_path = '../data/external/' +\
    'worldcitiespop.txt'

def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_2=code.upper()).name.lower()
    except KeyError:
        return 'unknown'
    
city_locations = pd.read_csv(city_locations_path, encoding = "ISO-8859-1").rename(columns={'City': 'city'})
city_locations = city_locations[city_locations['Population'] > 0] # filter for populated cities
city_locations['country'] = city_locations['Country'].apply(get_country_name)
print('Shape: {}'.format(city_locations.shape))
city_locations.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape: (47980, 8)


,Country,city,AccentCity,Region,Population,Latitude,Longitude,country
6,ad,andorra la vella,Andorra la Vella,7.0000,"20,430.0000",42.5000,1.5167,andorra
20,ad,canillo,Canillo,2.0000,"3,292.0000",42.5667,1.6000,andorra
32,ad,encamp,Encamp,3.0000,"11,224.0000",42.5333,1.5833,andorra
49,ad,la massana,La Massana,4.0000,"7,211.0000",42.5500,1.5167,andorra
53,ad,les escaldes,Les Escaldes,8.0000,"15,854.0000",42.5000,1.5333,andorra


In [348]:
earthquake_locations_path = '../data/external/' +\
    'earthquakes_affected_locations/earthquakes_affected_locations.csv'

earthquake_locations = pd.read_csv(earthquake_locations_path, encoding = "ISO-8859-1").drop('country', axis=1)
earthquake_locations['city'] = earthquake_locations['city'].apply(lambda c: c.lower())
print('Shape: {}'.format(earthquake_locations.shape))
earthquake_locations.head()

Shape: (2719, 12)


,id,city,latitude,longitude,geoname,year,month,day,Totaldeaths,Totalaffected,Totaldamage000US,insured_losses
0,1,takhar,36.7000,69.8000,NaN,1992,5,20,14,0,0,0
1,2,jowzan,36.7500,66.0000,Jowzjan,1994,5,1,160,100330,0,0
2,2,mazar-i-sharif,36.7500,67.0000,mazar-i-sharif,1994,5,1,160,100330,0,0
3,2,balkh,36.7501,66.8997,balkh,1994,5,1,160,100330,0,0
4,2,termez,37.2242,67.2783,NaN,1994,5,1,160,100330,0,0


In [313]:
category_cols = ['Region', 'Income group', 'Lending category']
class_path = '../data/external/' +\
    'class.csv'
world_bank_class = pd.read_csv(class_path, encoding = "ISO-8859-1", skiprows=[1])\
    .drop(['Unnamed: {}'.format(i) for i in [0,1,4]] + ['Code', 'Other'], axis=1)\
    .rename(columns={'Economy': 'country'})
    
le = LabelEncoder()
world_bank_class[category_cols] = world_bank_class[category_cols]\
    .apply(lambda c: le.fit_transform(c.astype(str)))
world_bank_class['country'] = world_bank_class['country'].apply(lambda c: str(c).lower())
world_bank_class.head()

,country,Region,Income group,Lending category
0,afghanistan,5,1,3
1,albania,1,3,2
2,algeria,3,3,2
3,american samoa,0,3,0
4,andorra,1,0,0


In [301]:
inflation_path = '../data/external/' +\
    'inflation.csv'
inflation = pd.read_csv(inflation_path, encoding = "ISO-8859-1", skiprows=[1]).rename(columns={'Country Name': 'country'})\
    .drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 61'], axis=1).set_index('country').fillna(0)

inflation = inflation[[c for c in inflation.columns if int(c) > 1992 & int(c) < 2014]].reset_index()
inflation = pd.melt(inflation, id_vars=['country'], value_vars=[str(y) for y in range(1993,2014)])\
    .rename(columns={'variable': 'year', 'value': 'inflation'})
inflation['country'] = inflation['country'].apply(lambda c: str(c).lower())
inflation.head()

,country,year,inflation
0,afghanistan,1993,0.0000
1,angola,1993,"1,379.4143"
2,albania,1993,85.0048
3,andorra,1993,0.0000
4,arab world,1993,9.3703


In [302]:
tot_path = '../data/external/' +\
    'terms_of_trade.csv'
terms_of_trade = pd.read_csv(tot_path, encoding = "ISO-8859-1", skiprows=[1]).rename(columns={'Country Name': 'country'})\
    .drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 61'], axis=1).set_index('country').fillna(0)

terms_of_trade = terms_of_trade[[c for c in terms_of_trade.columns if int(c) > 1992 & int(c) < 2014]].reset_index()
terms_of_trade = pd.melt(terms_of_trade, id_vars=['country'], value_vars=[str(y) for y in range(1993,2014)])\
    .rename(columns={'variable': 'year', 'value': 'terms_of_trade'})
terms_of_trade['country'] = terms_of_trade['country'].apply(lambda c: str(c).lower())
terms_of_trade.head()

,country,year,terms_of_trade
0,afghanistan,1993,0.0000
1,angola,1993,0.0000
2,albania,1993,0.0000
3,andorra,1993,0.0000
4,arab world,1993,0.0000


In [370]:
imports_path = '../data/external/' +\
    'imports.csv'
imports = pd.read_csv(imports_path, encoding = "ISO-8859-1", skiprows=[1]).rename(columns={'Country Name': 'country'})\
    .drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1).set_index('country').fillna(0)

imports = imports[[c for c in imports.columns if int(c) > 1992 & int(c) < 2014]].reset_index()
imports = pd.melt(imports, id_vars=['country'], value_vars=[str(y) for y in range(1993,2014)])\
    .rename(columns={'variable': 'year', 'value': 'imports'})
imports['country'] = imports['country'].apply(lambda c: str(c).lower())
imports.head()

,country,year,imports
0,afghanistan,1993,0.0000
1,angola,1993,55.3903
2,albania,1993,62.2925
3,andorra,1993,0.0000
4,arab world,1993,31.9900


In [372]:
exports_path = '../data/external/' +\
    'exports.csv'
exports = pd.read_csv(exports_path, encoding = "ISO-8859-1", skiprows=[1]).rename(columns={'Country Name': 'country'})\
    .drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1).set_index('country').fillna(0)

exports = exports[[c for c in exports.columns if int(c) > 1992 & int(c) < 2014]].reset_index()
exports = pd.melt(exports, id_vars=['country'], value_vars=[str(y) for y in range(1993,2014)])\
    .rename(columns={'variable': 'year', 'value': 'exports'})
exports['country'] = exports['country'].apply(lambda c: str(c).lower())
exports.head()

,country,year,exports
0,afghanistan,1993,0.0000
1,angola,1993,53.9033
2,albania,1993,15.4225
3,andorra,1993,0.0000
4,arab world,1993,28.5362


In [487]:
%run ../src/models/section_series.py
annual_frames = []
le = LabelEncoder()
for year in range(1992, 1993):
    print('Loading data for year {}'.format(year))
    try:
#         annual_frames.append(pd.read_csv('../data/processed/section_series_small/{}.csv'.format(year)).set_index(['year','city']))
        raise FileNotFoundError()
    except FileNotFoundError:
        earthquake_locations_in_year = earthquake_locations[earthquake_locations.year == year]
        earthquake_cities_in_year = earthquake_locations_in_year['city'].unique()
        df = pd.merge(city_locations, earthquake_locations_in_year, on='city', how='left')
        df = df.fillna(0).reset_index().groupby('city').agg({
            'Totaldeaths': 'sum',
            'Totalaffected': 'sum',
            'Totaldamage000US': 'sum',
            'insured_losses': 'sum',
            'Latitude': 'first',
            'Longitude': 'first',
            'country': 'first',
            'city': 'first'
        }).set_index('city').rename(columns={
            'Latitude': 'latitude',
            'Longitude': 'longitude',
            'Totaldeaths': 'total_deaths',
            'Totalaffected': 'total_affected',
            'Totaldamage000US': 'total_damage_usd'
        }).reset_index()
        df['year'] = year
        df = df.merge(world_bank_class, on='country', how='left')
        df = df.merge(terms_of_trade, on=['year','country'], how='left')
        df = df.merge(inflation, on=['year','country'], how='left')
        df = df.merge(imports, on=['year','country'], how='left')
        df = df.merge(exports, on=['year','country'], how='left')
        df['earthquake_dummy'] = df.set_index('city').index.map(lambda c: c in earthquake_cities_in_year)
        series_loader = SectionSeriesLoader(
            img_shape=(200, 200),
            start_year=year,
            end_year=year,
            SRC_PATH='../data/raw/Version_4_DMSP-OLS_Nighttime_Lights_Time_Series/*.npz')
        target_coords_list = [{'lat': r[0],'lng': r[1]} for r
            in df[['latitude', 'longitude']].values]
        df['mean_luminosity'] = series_loader.load_multiple_means(target_coords_list).reshape(-1)
        df = df.reset_index().set_index(['year','city']).drop('index', axis=1)
        df['country'] = le.fit_transform(df['country'])
        df.to_csv('../data/processed/section_series_small/{}.csv'.format(year))
        annual_frames.append(df)
        
panel = xr.Dataset.from_dataframe(pd.concat(annual_frames))
panel.to_dataframe().to_csv('../data/processed/section_series_small/panel.csv')
# # del annual_frames
panel

Loading data for year 1992
['../data/raw/Version_4_DMSP-OLS_Nighttime_Lights_Time_Series/F101992.v4b_web.stable_lights.avg_vis.tif.npz']
Loading file ../data/raw/Version_4_DMSP-OLS_Nighttime_Lights_Time_Series/F101992.v4b_web.stable_lights.avg_vis.tif.npz


/usr/local/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


<xarray.Dataset>
Dimensions:           (city: 43183, year: 1)
Coordinates:
  * year              (year) int64 1992
  * city              (city) object 0 'a' 'a coruna' 'a dos cunhados' ...
Data variables:
    total_deaths      (year, city) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    latitude          (year, city) float64 18.79 63.97 43.37 39.15 55.04 ...
    total_affected    (year, city) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    longitude         (year, city) float64 100.7 10.22 -8.407 -9.297 9.418 ...
    insured_losses    (year, city) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    country           (year, city) int64 182 137 170 147 49 49 69 177 49 58 ...
    total_damage_usd  (year, city) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Region            (year, city) float64 0.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 ...
    Income group      (year, city) float64 3.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Lending category  (year, city) float64 2.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    te

In [466]:
port_au_prince = panel.sel(city='port-au-prince').to_dataframe().fillna(0).drop(['city','latitude','longitude'],axis=1)
port_au_prince

,total_deaths,total_affected,insured_losses,country,total_damage_usd,Region,Income group,Lending category,terms_of_trade,inflation,imports,exports,earthquake_dummy,mean_luminosity
year,,,,,,,,,,,,,,
1992,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.2728
1993,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.2717
1994,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.2742
1995,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.4103
1996,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.4128
1997,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.3114
1998,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.3261
1999,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.3419
2000,0.0000,0.0000,0.0000,80,0.0000,2.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,False,0.3235


In [476]:
%%R -i port_au_prince
data <- port_au_prince
data$mean_luminosity <- log10(data$mean_luminosity*100)
data$earthquake_dummy <- as.numeric(data$earthquake_dummy)
lm(mean_luminosity ~ ., data)
data

     total_deaths total_affected insured_losses country total_damage_usd Region
1992            0              0          0e+00      80            0e+00      2
1993            0              0          0e+00      80            0e+00      2
1994            0              0          0e+00      80            0e+00      2
1995            0              0          0e+00      80            0e+00      2
1996            0              0          0e+00      80            0e+00      2
1997            0              0          0e+00      80            0e+00      2
1998            0              0          0e+00      80            0e+00      2
1999            0              0          0e+00      80            0e+00      2
2000            0              0          0e+00      80            0e+00      2
2001            0              0          0e+00      80            0e+00      2
2002            0              0          0e+00      80            0e+00      2
2003            0              0        

In [472]:
%%R -i port_au_prince
data <- port_au_prince
data$mean_luminosity <- log10(data$mean_luminosity)
require('dynlm')
# print(data)
data <- data.matrix(data)
dfm <- dynlm(data ~ L(data, 1) + L(data, 2), start=c(1995, 1), end=c(2013, 2))
print(dfm)


Error in terms.formula(formula) : '.' in formula and no 'data' argument


/usr/local/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in terms.formula(formula) : '.' in formula and no 'data' argument

  warnings.warn(x, RRuntimeWarning)


In [37]:
pred = linear_tree_booster.predict(test_data)
test_ae = test_data.get_label() - pred
test_wae = (test_data.get_label() - pred) * test_data.get_label()
test_mse = mean_squared_error(test_data.get_label(), pred)
true_mean = test_data.get_label().mean()